# Toplanan verilerden model eğitme

In [1]:
import numpy as np
import pandas as pd
import json

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding, CuDNNGRU
from tensorflow.keras.optimizers import Adam

2022-05-14 13:47:55.863137: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-14 13:47:55.863167: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
df = pd.read_csv('butunYorumlarSon.csv', sep = ',')
df.head()

,Unnamed: 0,tarih,puan,yorum,yeni_yorum
0,0,2021-10-19,5,Renkli çıktı da renkler gayet canlı çıkıyor. K...,renkli çıktı renkler gayet canlı çıkıyor kutu ...
1,1,2021-09-12,5,Ürün ile birlikte gelen evraklar ile kurulumu ...,ürün birlikte gelen evraklar kurulumu basit ko...
2,2,2021-12-23,5,Çok iyi bir yazıcı kurulumu mürekkebi koyması ...,iyi bir yazıcı kurulumu mürekkebi koyması kola...
3,3,2022-03-19,5,Kablosuz bağlantısı ile telefondan çikti ları ...,kablosuz bağlantısı telefondan çikti ları alab...
4,4,2022-03-19,5,Kurulumu basit ve hızlı.Güzel tasarım.Kullanım...,kurulumu basit hızlıgüzel tasarımkullanımı kol...


In [3]:
df = df.dropna()

In [4]:
target = df['puan'].values.tolist()
data = df['yeni_yorum'].values.tolist()

In [5]:
data[0:5]

['renkli çıktı renkler gayet canlı çıkıyor kutu içinden büyük siyah küçük renkli mürekkep çıkıyor mürekkeplerin dolumu yazıcının kurulumu gayet basit marka yazıcının sosyal medyada videolari var onları izleyerek herşey basit oluyor uzun süre problem çıkarmadan kullanabilecek kalite bir yazıcı',
 'ürün birlikte gelen evraklar kurulumu basit kolay bir şekilde yapılabiliyor kurulum bittikten sonra sn sonra baskı alınabilir fotoğraf baskısı gayet net güzel tavsiye ederim uygulama sayesinde ayrıca tarama yapıp telefona yüklüyor',
 'iyi bir yazıcı kurulumu mürekkebi koyması kolay kitapçığı bakarak yaparsanız hiçbir sorun yaşamazsınız kurulumunu yaptıktan sonra wifi ye bağlanıp cihazları bağlayana kadar kısık seste bip sesi geliyor geçiyor adet renkli ml tam dolduruyor tankını siyah olanda ml adet geliyor tam tankıyla aynı ml a kağıdına fotoğraf çıktısı aldım onda bile güzel çıktı kişiler tam kuramamış sanırım sorun yaşamışlar yorumlarda okuduğum zaman hiçbir sorun yaşamadım kurulumu herşeyi 

In [6]:
cutoff = int(len(data) * 0.85)
x_train, x_test = data[:cutoff], data[cutoff:]
y_train, y_test = target[:cutoff], target[cutoff:]

In [7]:
len(x_train)

28942

In [8]:
len(data)

34050

In [9]:
with open('tokenizer.txt','r') as f:
    s = f.read()
    s = s.replace("'", '"')

In [10]:
#s

In [11]:
with open('tokenizer.json') as json_dosyasi:
    json_tokenizer = json.load(json_dosyasi)

In [12]:
json_tokenizer['ben']

22

In [13]:
def tokenlestir(yorumListesi):
    y_yorumlar = []
    for yorum in yorumListesi:
        y_yorum = []
        for kelime in yorum.split():
            if (len(y_yorum) < 50 and kelime in json_tokenizer):
                y_yorum.append(json_tokenizer[kelime])
                
        if (len(y_yorum) < 50):
            sifirlar = list(np.zeros(50 - len(y_yorum), dtype = int))
            y_yorum = sifirlar + y_yorum
            
        y_yorumlar.append(y_yorum)
    return np.array(y_yorumlar, dtype = np.dtype(np.int32))

In [14]:
egitim_kume = tokenlestir(x_train)

In [15]:
egitim_kume[122]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0, 4927, 6600,    6,    3,    1,    2, 2023,
        311, 2490,  172,  277,  202,   43,    7,   22,    1,    2,   25,
         32, 1451,   43,  768,    4,  447], dtype=int32)

In [16]:
test_kume = tokenlestir(x_test)

In [17]:
test_kume[122]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0, 3086,   37,    1,   28, 7160, 4956,  297,
       1871, 1362, 1108, 1871,  216,  216,  310,    1, 1736, 5065,   67,
          1,   67,    1,   67,    1,  912], dtype=int32)

In [19]:
model = Sequential()

embedding_size = 50

model.add(Embedding(input_dim=10001,
                    output_dim=embedding_size,
                    input_length=50,
                    name='embedding_layer'))

model.add(GRU(units=16, return_sequences=True))
model.add(GRU(units=8, return_sequences=True))
model.add(GRU(units=4))
model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(lr=1e-3)

model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 50, 50)            500050    
_________________________________________________________________
gru_3 (GRU)                  (None, 50, 16)            3216      
_________________________________________________________________
gru_4 (GRU)                  (None, 50, 8)             600       
_________________________________________________________________
gru_5 (GRU)                  (None, 4)                 156       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5         
Total params: 504,027
Trainable params: 504,027
Non-trainable params: 0
_________________________________________________________________


In [20]:
n_y_train = np.array(y_train)

In [21]:
model.fit(egitim_kume, n_y_train, epochs = 5, batch_size = 25)

Epoch 1/5
 112/1158 [=>............................] - ETA: 1:00 - loss: -5.9630 - accuracy: 0.0186

KeyboardInterrupt: 